In [1]:
from flask import Flask, request, jsonify
from flask_restful import Resource, Api, reqparse
from flask_jwt import JWT
from flask_jwt import jwt_required
from security import authenticate, identity
from user import UserRegister
import sqlite3


app = Flask(__name__)
app.secret_key = 'tym'
api = Api(app)

jwt = JWT(app, authenticate, identity) # new endpoint /auth

class Item(Resource):
    @jwt_required()
    def get(self, name):
        connection = sqlite3.connect('data.db')
        cursor = connection.cursor()
        
        query = "SELECT * FROM items WHERE name =?"
        result = cursor.execute(query, (name,))
        row = result.fetchone()
        connection.close()
        
        if row:
            return {'item': {'name': row[1],
                            'price': row[2]}}
        return {'message', 'item not found'}
        
        
    def post(self, name):
        data = Item.validate_data(['price'],'This field cannot be left blank').parse_args()
        
        if Item.find_by_name(name) == True:
            return {'message': "'{}' already exists".format(name)}
        
        connection = sqlite3.connect('data.db')
        cursor = connection.cursor()
        query = "INSERT INTO items VALUES(NULL, ?, ?)"
        cursor.execute(query, (name, data['price']))
        connection.commit()
        if connection.total_changes > 0:
            connection.close()
            return {'message': "'{}' was added with a price of {}".format(name,data['price'])}, 201
        return {'message': "'{}' was not added".format(name)}, 400
            
    def delete(self, name):
        connection = sqlite3.connect('data.db')
        cursor = connection.cursor()
        query = "DELETE FROM items WHERE name = ?"
        cursor.execute(query, (name,))
        connection.commit()
        if connection.total_changes > 0:
            connection.close()
            return {'message': "'{}' item deleted".format(name)}
        return {'message': "'{}' item was not deleted".format(name)}
    
    def put(self, name):
        data = Item.validate_data(['price'],'This field cannot be left blank').parse_args()
        
        connection = sqlite3.connect('data.db')
        cursor = connection.cursor()
        query = "UPDATE items SET price = ? WHERE name = ?"
        cursor.execute(query, (data['price'], name))
        connection.commit()
        if connection.total_changes > 0:
            connection.close()
            return {'message': "'{}' was updated by {}".format(name,data['price'])}, 201
        return {'message': "'{}' was not updated".format(name)}, 400
    
    @classmethod
    def find_by_name(cls, name):
        connection = sqlite3.connect('data.db')
        cursor = connection.cursor() 
        query = "SELECT * FROM items WHERE name = ?"
        result = cursor.execute(query, (name,))
        row = result.fetchone()
        connection.close()
        return True if row else False        
    
    @classmethod
    def validate_data(self, fields, message):
        parser = reqparse.RequestParser()  
        for field in fields:
            parser.add_argument(field, 
                            type=str, 
                            required=True, 
                            help=message
        )
        return parser
    
class ItemList(Resource):
    def get(self):
        return {'items': items}

api.add_resource(Item, '/item/<string:name>') # http://127.0.0.1/itemt/<item name>
api.add_resource(ItemList, '/items')
api.add_resource(UserRegister, '/register')
app.run(port=5001)

 * Running on http://127.0.0.1:5001/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Jan/2018 15:58:57] "POST /item/lamp HTTP/1.1" 200 -
